### Work with ML data

In [1]:
import plotly.express as px
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import numpy as np
from sklearn.manifold import TSNE

In [2]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

### Step 1: Data Preprocessing

In [3]:
df = pd.read_csv('ml_data.csv')  
text_messages = df['text']  

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words and word not in string.punctuation]
    # tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() and word not in string.punctuation]
    return ' '.join(tokens)

df['cleaned_text'] = df['text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mchomak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mchomak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mchomak\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Mchomak\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


### Step 2: Feature Extraction

In [4]:
stop_words=list(stop_words)

tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words=stop_words) 
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_text'])

### Step 3: Train DBSCAN model


In [5]:
for eps in range(1, 110, 5):  # Adjust the range as needed
    eps=eps/100
    dbscan = DBSCAN(eps=eps, min_samples=5)
    cluster_labels = dbscan.fit_predict(tfidf_matrix)
    if len(np.unique(cluster_labels)) > 1:  # silhouette_score requires at least 2 clusters
        silhouette_avg = silhouette_score(tfidf_matrix, cluster_labels)
        print(f"Epsilon: {eps}, Silhouette Score: {silhouette_avg}")
    else:
        print(f"Epsilon: {eps}, Silhouette Score: N/A (Single Cluster)")

Epsilon: 0.01, Silhouette Score: -0.16521308480305497
Epsilon: 0.06, Silhouette Score: -0.16521308480305497
Epsilon: 0.11, Silhouette Score: -0.16521308480305497
Epsilon: 0.16, Silhouette Score: -0.16521308480305497
Epsilon: 0.21, Silhouette Score: -0.16521308480305497
Epsilon: 0.26, Silhouette Score: -0.16521308480305497
Epsilon: 0.31, Silhouette Score: -0.16521308480305497
Epsilon: 0.36, Silhouette Score: -0.16521308480305497
Epsilon: 0.41, Silhouette Score: -0.16521308480305497
Epsilon: 0.46, Silhouette Score: -0.16521308480305497
Epsilon: 0.51, Silhouette Score: -0.16521308480305497
Epsilon: 0.56, Silhouette Score: -0.16521308480305497
Epsilon: 0.61, Silhouette Score: -0.16521308480305497
Epsilon: 0.66, Silhouette Score: -0.16521308480305497
Epsilon: 0.71, Silhouette Score: -0.16521308480305497
Epsilon: 0.76, Silhouette Score: -0.15562996742108248
Epsilon: 0.81, Silhouette Score: -0.1621166126850712
Epsilon: 0.86, Silhouette Score: -0.16343128592699774
Epsilon: 0.91, Silhouette Sco

In [6]:
eps = 0.97
min_samples = 5 

dbscan = DBSCAN(eps=eps, min_samples=min_samples)
df['cluster'] = dbscan.fit_predict(tfidf_matrix)

### Step 4: Dimensionality Reduction with t-SNE

In [7]:
tsne = TSNE(n_components=2, random_state=42)
reduced_features_tsne = tsne.fit_transform(tfidf_matrix.toarray())

### Step 5: visualization with plotly

In [8]:
df['tsne1'] = reduced_features_tsne[:, 0]
df['tsne2'] = reduced_features_tsne[:, 1]

# Create a scatter plot with hover text
fig = px.scatter(df, x='tsne1', y='tsne2', color='cluster',
                 title='Text Clustering Visualization (DBSCAN + t-SNE)',
                 labels={'tsne1': 't-SNE Component 1', 'tsne2': 't-SNE Component 2'},
                 template='plotly', color_continuous_scale='viridis',
                 hover_name=df['text'], hover_data=['cluster'],
                width=1000, height=1000)

# Customize the hover text
fig.update_traces(textposition='top center')

# Show the interactive plot
fig.show()

In [10]:
import plotly.graph_objects as go

# Assuming 'data' DataFrame with 'cluster' column

# Create a scatter plot with hover text and legend using go.Figure
fig = go.Figure()

for cluster in df['cluster'].unique():
    if cluster!=-1:
        cluster_data = df[df['cluster'] == cluster]
        fig.add_trace(go.Scatter(
            x=cluster_data['tsne1'],
            y=cluster_data['tsne2'],
            mode='markers',
            marker=dict(color=cluster, size=10, line=dict(color='black', width=2)),  # Customize marker size and line properties
            text=cluster_data['text'],
            name=f'Cluster {cluster}'
        ))

# Customize layout
fig.update_layout(
    # title='Text Clustering Visualization (DBSCAN + t-SNE)',
    xaxis_title='t-SNE Component 1',
    yaxis_title='t-SNE Component 2',
    legend=dict(title='Clusters', orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0.01),
    hoverlabel=dict(bgcolor="white", font_size=12, namelength=-1),
    width=900, height=900
)

# Show the interactive plot
fig.show()


### Step 6: Watch all clasters text

In [ ]:
df = df.sort_values(by='cluster')

In [ ]:
df['cluster'].unique()

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=int64)

In [ ]:
# Print text messages in each cluster
for cluster_id in df['cluster'].unique():
    if cluster_id!=-1:
        cluster_data = df[df['cluster'] == cluster_id]['text']
        print(f"\nCluster {cluster_id}:\n")
        for text in cluster_data:
            print(text)
        print("="*50)


Cluster 0:

тогда еще ладно
тогда бб
Ладно
ладно у него сам спрошу
ладно бб
Все бб
Ладно
Ладно
Ладно я точно теперь спать бб

Cluster 1:

+
и как ?
Как
(((((
.
А не
Ну ничего
я x1/5
А
Ты где ?
да уж
Да
да
Да все не надо
Да уж
ну еще бы
ну да +-
или как ?
???
Что то
Ты о чем
во
Так что можно
Ну все
У меня так теперь
Все тогда
я в 11
Так
😬
да
Да
да
Да
Ты сам где сейчас?
1000
У меня так
а
У меня тоже все
Или как
12к
конечно
...
А ты где сейчас
я об этом же
так не
и еще лучше будет
+
Не
Что тоже так себе
Но все потом да потом
Да
А была 4 про
А
можно
Хорошо
И что там?
о
да

Cluster 2:

Ага хаха
Ага
ага
хаха
ага
Ага
Ага
Ага
Ага
ага
ты тоже можешь хаха
Хаха ну да
Так и сделаю хаха
Ага
хаха
ага

Cluster 3:

Давай фокус
Окей
давай
Окей, давай
Окей
давай
давай
Окей
Окей
Давай
Окей
Окей
Не, давай сегодня скип

Cluster 4:

Бывает
Апхапхап
Апхапхап
бывает
да бывает
бывает
Апхапхап, бывает

Cluster 5:

Да бро
бро тут только факты
обращайся бро
факты
Да факты
Факты
да факты
Завтра разберу бро
Спасиб